<a href="https://colab.research.google.com/github/Aditya-Shandilya1182/neo/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [3]:
#embd -> DIM
#hidden_dim -> FFN_DIM
#n_head -> N_HEADS
#head_dim -> HEAD_DIM

In [2]:
class MLP(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.w1 = nn.Linear(config.embd, config.hidden_dim, bias=False)
    self.w2 = nn.Linear(config.hidden_dim, config.embd, bias=False)
    self.w3 = nn.Linear(config.embd, config.hidden_dim, bias=False)

  def forward(self, x):
    x_w1 = self.w1(x)
    x_w3 = self.w3(x)
    x = F.silu(x_w1) * x_w3
    x = self.w2(x)
    return x

In [ ]:
class Attention(nn.Module):
  def __init__(self, config):
    super().__init__()

    self.wq = nn.Linear(config.embd, config.n_heads * config.head_dim, bias=False)
    self.wk = nn.Linear(config.embd, config.n_kv_head * config.head_dim, bias=False)
    self.wv = nn.Linear(config.embd, config.n_kv_head * config.head_dim, bias=False)
    self.wo = nn.Linear(config.n_heads * config.head_dim, config.embd, bias=False)

    self.cache_k = torch.zeros(
        (
            config.batch_size,
            config.seq_len,
            config.n_kv_head,
            config.head_dim
        )
    )

    self.cache_v = torch.zeros(
        (
            config.batch_size,
            config.seq_len,
            config.n_kv_head,
            config.head_dim
        )
    )

    self.n_heads = config.n_heads
    self.head_dim = config.head_dim
    self.n_kv_head = config.n_kv_head

  def forward(self, x, start_pos, freqs_cis, mask):
    batch_sz, seqlen, _ = x.shape

    queries = self.wq(x)
    keys = self.wk(x)
    values = self.wv(x)

    queries = queries.view(batch_sz, seqlen, self.n_heads, self.head_dim)
    keys = keys.view(batch_sz, seqlen, self.n_kv_head, self.head_dim)
    values = values.view(batch_sz, seqlen, self.n_kv_head, self.head_dim)

